In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
%matplotlib inline

# 한글대응
from matplotlib import rc
%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False
rc('font',family='Malgun Gothic')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from konlpy.tag import Okt

In [5]:
shampoo_data = pd.read_csv('./shampoo_final.csv')
shampoo_data.head()

,리뷰,평점,긍정부정,리뷰길이
0,진짜 머리 감고 나면 너무 너무 간지러워 요상쾌함이 오래가지도 못하고 냄새도 빨리 나요,1,n,52
1,비듬 생겼어요 어쩐지간지럽더라 개나 샀는데 누구 주지도 못하고 아오,1,n,50
2,뚜껑 펌프 고장 났네요 아무리 눌러도 안 나와서 뚜껑 열고 사용하다 방치했어요,1,n,40
3,머리카락에 부담이 안 되고 머리 피부에 부담이 안 되니 좋죠,1,n,27
4,샴푸는 사용해보고 살수가 없으니 오랜 기간 동안 고민했고 리뷰들도 꼼꼼히 보았는데 ...,1,n,510


- 감성 분석용 말뭉치를 가져와 모델에게 학습시킨다
- 샴푸 데이터를 정제
    - 형태소 분석한 이후 알맞지 않다고 생각되는 부분 수정 혹은 삭제?
    - 조사, 접속사, 접미사 형태소 분석 중 불필요한 부분


## 데이터 정제

In [6]:
shampoo_okt = shampoo_data.copy()

okt = Okt()

In [7]:
all_words = []
for idx, row in shampoo_okt.iterrows():
    
    sentence = okt.pos(row['리뷰'],stem=True)
    words = []
    for each in sentence:
        if each[1] not in ['Josa','Conjunction','Suffix']:
            words.append(each[0])
    all_words.append(words)

In [8]:
for idx, row in shampoo_okt.iterrows():
    print(row['리뷰'])
    print(all_words[idx])
    print()

진짜 머리 감고 나면 너무 너무 간지러워 요상쾌함이 오래가지도 못하고 냄새도 빨리 나요
['진짜', '머리', '감고', '나다', '너무', '너무', '간지럽다', '요', '상쾌하다', '오래', '가지', '못', '냄새', '빨리', '나요']

비듬 생겼어요 어쩐지간지럽더라 개나 샀는데 누구 주지도 못하고 아오
['비듬', '생기다', '어쩐지', '간지럽다', '개', '사다', '누구', '주지', '못', '아오']

뚜껑 펌프 고장 났네요 아무리 눌러도 안 나와서 뚜껑 열고 사용하다 방치했어요
['뚜껑', '펌프', '고장', '나다', '아무리', '누르다', '안', '나오다', '뚜껑', '열다', '사용', '하다', '방치', '하다']

머리카락에 부담이 안 되고 머리 피부에 부담이 안 되니 좋죠
['머리카락', '부담', '안', '되다', '머리', '피부', '부담', '안', '되다', '좋다']

샴푸는 사용해보고 살수가 없으니 오랜 기간 동안 고민했고 리뷰들도 꼼꼼히 보았는데 저와 같은 두피 고민이 해결되었다는 내용이 꽤 있어 구매를 결심하게 됐습니다 그런데 일단 결과적으로 저에게는 너무나 안 맞아서 속상하게도 하루도 더 사용할 수가 없네요 제두피 상태는 수분 부족 지성으로 알고 있어 요건조해서 가렵고 각질이 일어나지만 샴푸를 하루만 걸러도 기름지고 답답하고 이틀이 되면 모근 부분이 자극이 되어 아프거든요제가 메디올 샴푸를 쓰고 느낀 점은 세정력이 부족하고 유분이 너무 많다는 거예요 거품은 적당히 풍부하고 머릿결은 개선되는 듯 보였지만 다음날 자고 일어났을 때 베개가 정말 눈밭이 되어 있는 걸 보고 자기 혐오를 느끼고 말았습니다 그리고 사실 첫 펌핑했을 때 향도 조금 거부감이 들었어요 개나 사버렸으니 잘 맞았다면 정말 좋았겠지만 아무래도 당근행이 될 거 같아 요광고도 요즘 정말 많이 보이고 찬양 글도 워낙 많던데 당연히 저처럼 안 맞는 사람도 꽤 있겠죠 누군가에겐 참고가 되었으면 좋겠습니다
['샴푸', '사용', '해보다


이걸로 감고 각질 바다가 됬네요 쓰레기 제품 광고비만 엄청 써서 팔지 말고 양심껏 팝시다 만족도 마이나스 줘도 모자라네 요 속지 마세요
['이', '걸', '감고', '각질', '바다', '되다', '쓰레기', '제품', '광고', '비만', '엄청', '써다', '팔지', '양심', '껏', '파다', '만족도', '마이나스', '주다', '모자라다', '요', '속지', '말다']

다른 분들은 모르겠으나 저는 효과가 없네요 뒷머리에 뾰루지가 많이 나서 써봤는데 효과가 없어서 주 쓰고 끊었습니다 다만 그냥 샴푸로서의 느낌은 좋아요
['다른', '분', '모르다', '저', '효과', '없다', '뒷', '머리', '뾰루지', '많이', '나서다', '써다', '보다', '효과', '없다', '주', '쓰다', '끊다', '다만', '그냥', '샴푸', '로서', '느낌', '좋다']

광고에서 말하는 효과는 별로 못 봤네요 요히려 악화됨 나한테 안 맞는 건지 환불도 안 될 거고 그냥 주변에 나눠줘야 할 듯
['광고', '말', '하다', '효과', '별로', '못', '보다', '요', '히', '려', '악화', '되다', '나', '안', '맞다', '건지다', '환불', '안', '되다', '거고', '그냥', '주변', '나누다', '하다', '듯']

좋다고 해서 구매한 이후 한 달을 넘게 계속 써봤는데 두피 각질이 심해졌어 요저랑은 맞지 않나 봅니다 통구매해서 통은 아직 뜯지도 않았고 이제 정도 썼는데 간단히 손빨래할 때 쓰고 있습니다
['좋다', '하다', '구매', '이후', '하다', '달', '넘다', '계속', '써다', '보다', '두피', '각질', '심해지다', '요', '저', '맞다', '않다', '보다', '통', '구매', '하다', '통', '아직', '뜯다', '않다', '이제', '정도', '써다', '간단하다', '손', '빨래', '하다', '때', '쓰다', '있다']

첨에는 효과 있는 듯 하나 반통쓴 지금으로

거품이 기존보다 발생기고 세정력 좋은 듯
['거품', '기존', '발생', '기고', '세', '정력', '좋다', '듯']

향이 나쁘지 않고 우선 열심히 사용해 봉께요
['향', '나쁘다', '않다', '우선', '열심히', '사용', '하다', '봉', '요']

배송이 좀 느리긴 했지만 상품은 만족합니다
['배송', '좀', '느리다', '하다', '상품', '만족하다']

다른 샴푸보다 자극이 덜한 것 같아서 좋아요
['다른', '샴푸', '자극', '덜하다', '것', '같다', '좋다']

사용하고 나면 모발에 볼륨이 조금 있는 것 같아요
['사용', '나다', '모발', '볼륨', '조금', '있다', '것', '같다']

소문 듣고 쓰봅니다 탈모에 도움되너나 믿고 리뷰 또 남길게요
['소문', '듣다', '쓰다', '보다', '탈모', '도움', '되다', '너', '믿다', '리뷰', '또', '남다']

조아조아조아도 아동조아
['조아', '조아', '좋다', '아동']

몇 년째 이것만 쓰고 있습니다
['몇', '년', '이', '것', '쓰다', '있다']

알고 쓰는 제품이라 구입했어요
['알', '쓰다', '제품', '구입', '하다']

잘 도착햇고 아직 사용 전이네요
['자다', '도착', '햇', '아직', '사용', '전이']

한번 써봐서 아직은 잘 모르겠어요
['한번', '써다', '보다', '아직', '은', '자다', '모르다']

전에 써봤는데 괜찮아서 재구매함
['전', '써다', '보다', '괜찮다', '재다', '함']

저한테는 잘 안 맞는 듯 너무 기름져요
['저', '한', '테', '자다', '안', '맞다', '듯', '너무', '기르다']

저는 이것만 씁니다 좋은 건 모르겠음
['저', '이', '것', '쓰다', '좋다', '건', '모르다']

상품은 괜찮지만 배달은 진짜 느려요
['상품', '괜찮다', '배달', '진짜', '느리다']

배송이 늦어서 취소하고 다른 거샀는데 왔네요 두고두고 써야

향이 센 건 못 쓰겠어서 사봤는데 장미향이나네요트리트먼트보다 향이 센 편이에요
['향', '세다', '건', '못', '쓰다', '사보다', '장미', '향', '네', '요트', '리트', '멀다', '트', '향', '세다', '편이']

향이 일단 너무 좋아용
['향', '일단', '너무', '좋다']

세정력과 향이 비추
['세', '정력', '향', '비추다']

다음에도 또 주문 힐께요
['다음', '또', '주문', '힐', '요']

괜찮은 거 같아 용아마동
['괜찮다', '거', '같다', '용다', '마동']

출산 후 머리가 많이 빠져 사봤는데 아직 효과는 모르겠네요 향은 괜찮습니다
['출산', '후', '머리', '많이', '빠지다', '사보다', '아직', '효과', '모르다', '향', '괜찮다']

향이 은은해 요지성이라면 오후에는 사알 짝 냄새올라올 수도 있을 것 같아요 그 외에는 자극 없고 좋습니다
['향', '은은하다', '요', '지성', '라면', '오후', '사', '알', '짝', '냄새', '올라오다', '수도', '있다', '것', '같다', '그', '외', '자극', '없다', '좋다']

잘 안 잠겼는지 샴푸가 흘렀어요급해서 그냥 쓰는데 앞으로 신경 써주셨으면 합니다
['자다', '안', '잠기다', '샴푸', '흐르다', '급하다', '그냥', '쓰다', '앞', '신경', '써주다', '하다']

오늘 처음 써봐서 아직 두피 가려움 등에 좋은지는 한 달 후 리뷰로 거품은 잘 나 여향도 괜춘
['오늘', '처음', '써다', '보다', '아직', '두피', '가려움', '등', '좋다', '하다', '달', '후', '리뷰', '거품', '자다', '나', '여', '향', '괜춘']

향이 좋고 거품이 잘나요 근데 장미향은 나지만 깜고 나면 장미향이 별로 안 나는 거 같아 요 꾸욱 펌핑하면 샴푸 양이 많이 나와서 살짝만 눌러도 되요
['향', '좋다', '거품', '자다', '근데', '장미', '향', '나

감으면 볼륨이 산다는 장점화한 느낌은 없어요
['감다', '볼륨', '산다', '장점', '느낌', '없다']

아들이 사용 중인데 여전히 많이 빠지고 있네요
['아들', '사용', '중', '여전하다', '많이', '빠지다', '있다']

아직 안 써봐서 모르겠는데 효과 좋으면 좋게 써여
['아직', '안', '써다', '보다', '모르다', '효과', '좋다', '좋다', '써다']

두피 가려운 것도 없고 무난하게 잘 써서 재구매해여
['두피', '가다', '것', '없다', '무난', '하다', '자다', '써다', '재다', '해']

요즘 머리가 너무 많이 빠지고 두피도 가려워서 유투버 추천 보고 구매해봤는데 좋았으면 해요
['요즘', '머리', '너무', '많이', '빠지다', '두피', '가다', '워', '유투', '버', '추천', '보고', '구매', '해봤다', '좋다', '해']

효과는 아직 잘 모르겠지만 꾸준히 써보려고 합니다
['효과', '아직', '자다', '모르다', '꾸준하다', '써다', '보다', '하다']

탈모 때문에 구입했는데 효과가 있는지는 잘 모르겠어요
['탈모', '때문', '구입', '하다', '효과', '있다', '자다', '모르다']

배송 빠르고 상태 양호
['배송', '빠르다', '상태', '양호']

무나 뮤난한 것 같아요
['무', '뮤', '난하다', '것', '같다']

잘 사용하고 있습니다
['자다', '사용', '있다']

거품안나 구요뻑뻑 하구별로 입니다
['거품', '안나', '구', '요', '뻑뻑', '하다', '별로', '이다']

잘 받았어요 좋타해서 함써보려고 주문했는데 저에게도 잘 맞았음 좋겠네요 잘 쓸게요
['자다', '받다', '좋다', '타', '해', '함', '써다', '보다', '주문', '하다', '저', '자다', '맞다', '좋다', '자다', '쓸다']

무난하게 쓰기 좋아요 아직까지 머리가 덜 빠진다던가 그런 건 잘 모르겠어요
['무난', '하다', '쓰기',

샴푸 중에 케라시스가 좋다는 말을 듣고 예전부터 계속 쓰는 중
['샴푸', '중', '케라시스', '좋다', '말', '듣다', '예전', '계속', '쓰다', '중']

염색을 자주 해서 주문했는데 효과가 있으면 좋겠네요
['염색', '자주', '하다', '주문', '하다', '효과', '있다', '좋다']

예전에 써봤던 제품에 라 다시 써 볼라고 그 매합니다
['예전', '써다', '보다', '제품', '다시', '써다', '볼라', '그', '매', '하다']

불우이웃돕기 하느라 대량 구매 햇어요 저렴하네요
['불우', '이웃', '돕기', '하다', '대량', '구매', '햇', '저렴하다']

대여자자 쓰고 있어 요 머리결이 한결 부드러요
['대', '여자', '쓰다', '있다', '요', '머리', '결', '한결', '부드럽다']

배송이 좀 느리긴 했지만 잘 쓰도록 하겠습니다
['배송', '좀', '느리다', '하다', '자다', '쓰다', '하다']

가격도 경제적이고 머릿결이 부드러워졌어요
['가격', '경제', '머릿결', '부드럽다']

후기 좋고 가성비도 좋은 듯해서 구매 햇어요
['후기', '좋다', '가성', '비도', '좋다', '듯', '하다', '구매', '햇']

다른 샴푸 쓰면 많이 가려워서 이것만 써요
['다른', '샴푸', '쓰다', '많이', '가다', '워', '이', '것', '써다']

저번에 샀는데 집에어딧는지 몰라 또 삼
['저번', '사다', '집에어딧는', '모르다', '또', '삼']

전체적으로 부족함 없이 무난한 것 같아요
['전체', '부족함', '없이', '무난', '것', '같다']

머릿결이 많이 상해서 사봤는데 그럭저럭
['머릿결', '많이', '상해', '사보다', '그럭저럭']

엘라스틴과 번갈아가며 사용하고 있어요
['엘라스', '틴', '번갈아', '가다', '사용', '있다']

저렴하고 좋아요쟁겨놓고 쓰네요
['저렴하다', '좋다', '쟁', '기다', '쓰다']

용량이 엉청 

늘 사용하는 거지만 맘에 쏙듬니다
['늘', '사용', '하다', '거지', '맘', '쏙듬니']

케라시스향이 좋아서 항상 씁니다
['케라시스', '향', '좋다', '항상', '쓰다']

저렴하게 잘 삿습니다 너무 좋아요
['저렴하다', '자다', '삿습니', '너무', '좋다']

누구나 다 아는 그 제품 싸게 잘 샀어요
['누구', '다', '알다', '그', '제품', '싸다', '자다', '사다']

착한 가격에 아주 만족하는 상품입니다
['착하다', '가격', '아주', '만족하다', '상품', '이다']

향도 강하지 않고 잘 사용하고 있어요
['향', '강하다', '않다', '자다', '사용', '있다']

매번 구매하고 있어요아주 만족합니다
['매번', '구매', '있다', '아주', '만족하다']

배송 빠르고 저렴하게 잘 샀어요
['배송', '빠르다', '저렴하다', '자다', '사다']

배송도 빠르고 넘 좋아요
['배송', '빠르다', '넘다', '좋다']

늘 쓰던 거회사에서 쓰는 거
['늘', '쓰다', '거', '회사', '쓰다', '거']

가성비 대비 좋은 것 같아요
['가성', '비', '대비', '좋다', '것', '같다']

갠찬아 요상큼하고 좋아요
['갠차다', '요', '상큼', '좋다']

향도 괜찮고 사용기한도 길어요포장에 신경을 써주세요 전혀 포장에 신경을 안 쓰셨네
['향', '괜찮다', '사용', '기한', '기다', '포장', '신경', '써주다', '전혀', '포장', '신경', '안', '쓸다']

잘 받았습니다 감사합니다
['자다', '받다', '감사하다']

늘 구매해서 사용하는 제품은 은한 향도 은은하고 저렴하게 구매해서 만족해요
['늘', '구매', '하다', '사용', '하다', '제품', '은한', '향', '은은하다', '저렴하다', '구매', '하다', '만족하다']

매년 구입해서 잘 사용하고 있습니다다음에 또 주문하겠습니다
['매년', '구입', '하다', '자다', '사용', '있다'

네 번째 구매엡니다 그냥 준수한 편이에 요마일드하고 가급적 마일드한 내용물이라 믿고 구매해서 사용합니다 다 만 두피가 눈에 뜨게 개선되거나 모발이 봏아지는 것 같지는 않아 요새 정력은 상대적으로 좋은 편입니다 자극도 없고
['네', '구', '매', '엡', '다', '그냥', '준수', '편이', '요', '마일', '드하', '가급적', '마', '일드', '내용물', '믿다', '구매', '하다', '사용', '하다', '다', '만', '두피', '눈', '뜨다', '개선', '되다', '모발', '봏아', '것', '같다', '않다', '요새', '정력', '상대', '좋다', '편입', '니', '자극', '없다']

신랑비듬 때문에 여러 제품 사용해보고 있는데 첫날 사용하고 효과가 있는 듯했습니다 근데 일쯤부터 다시 보이기 시작하네 요 입자는 그동안 보다 좀 작아지 것 같기는 합니다 좀 더 장기간 사용해봐야 확실할 것 같고 딸아이는 잘 맞지 않는다고 하네요 샴푸 성분이 잘 맞는지는 사람마다 다른 것 같습니다 함께 쓸 수 있는 샴푸를 찾고 싶네요
['신랑', '비듬', '때문', '여러', '제품', '사용', '해보다', '있다', '첫날', '사용', '효과', '있다', '듯', '하다', '근데', '일', '쯤', '다시', '보이다', '시작', '하다', '요', '입자', '그동안', '보다', '좀', '작아지다', '것', '같다', '하다', '좀', '더', '장기간', '사용', '해보다', '확실하다', '것', '같다', '딸아이', '자다', '맞다', '않다', '하다', '샴푸', '성분', '자다', '맞다', '사람', '다른', '것', '같다', '함께', '쓸다', '수', '있다', '샴푸', '찾다', '싶다']

아직 몇일 써보진 못했지만 제가 정말 지성 두피라 그런지 다음날 되어도 기름이 안진다보다는 조금 덜하다 인 게 나을 듯합니다 그러나 샴퓨만 써도 머리결이 부드러운 건 확실합니다제가 써본 탈모 샴

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
stop_words = ['요','개','니','조','랄','이','걸','…','저하','넘다','스럽다','님','데','것','거','음', '주정','한','테','겨','움','이다']

In [10]:
len(all_words)

10279

### 네이버 쇼핑 데이터

In [11]:
shopping_data = pd.read_csv('./naver_shopping.csv')

In [12]:
shopping_data.rename(columns={'5':'평점','배공빠르고 굿':'리뷰'},inplace=True)

In [13]:
shopping_data

,평점,리뷰
0,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
1,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
2,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
3,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
4,2,비추합니다 계란 뒤집을 때 완전 불편해요 ㅠㅠ 코팅도 묻어나고 보기엔 예쁘고 실용적...
...,...,...
199994,2,장마라그런가!!! 달지않아요
199995,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199996,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199997,5,넘이쁘고 쎄련되보이네요~


In [14]:
shopping_data['평점'].value_counts()

5    81176
2    63989
1    36048
4    18786
Name: 평점, dtype: int64

- 각 점수당 10000개씩

In [15]:
shampoo_data['평점'].value_counts()

5    2723
4    2705
3    2500
1    1183
2    1168
Name: 평점, dtype: int64

In [16]:
s_5 = shopping_data[shopping_data['평점'] == 5]
score_5 = list(np.random.choice(s_5['리뷰'], 10000, replace=False))
s_4 = shopping_data[shopping_data['평점'] == 4]
score_4 = list(np.random.choice(s_4['리뷰'], 10000, replace=False))
s_2 = shopping_data[shopping_data['평점'] == 2]
score_2 = list(np.random.choice(s_2['리뷰'], 10000, replace=False))
s_1 = shopping_data[shopping_data['평점'] == 1]
score_1 = list(np.random.choice(s_1['리뷰'], 10000, replace=False))

In [17]:
review = score_5 + score_4 + score_2 + score_1

In [18]:
2500 + 2705 + 1183 + 1168

7556

In [19]:
scores = []
for i in range(40000):
    if i < 10000:
        scores.append(5)
    elif i < 20000:
        scores.append(4)
    elif i < 30000:
        scores.append(2)
    else:
        scores.append(1)

In [20]:
len(review), len(scores)

(40000, 40000)

In [21]:
shopping = pd.DataFrame({'리뷰':review,'평점':scores})
shopping.head()

,리뷰,평점
0,깔끔해요. 데일리로 좋아요. 무게감은 가벼워요,5
1,버블시키면서 에빠니 드로퍼 샀어요.좋아요,5
2,맛난 팥 빙수용 팥 찾다가 구매했는데 리뷰보고 아무생각없이 산 것 치고는 너무 맛있...,5
3,실물로 그냥 봤을 때는 조금 허술해 보이고 조잡스러웠는데 조명만 껴고 보니 예뻐요....,5
4,좋아요. 음감도 좋고 아들 잘 치고 있어요,5


### 형태소 분석

In [123]:
shopping_words = []

for idx, row in shopping.iterrows():
    
    sentence = okt.pos(row['리뷰'],stem=True)
    words = []
    for each in sentence:
        if each[1] not in ['Josa','Conjunction','Suffix']:
            words.append(each[0])
    shopping_words.append(words)

### 말뭉치

In [124]:
shopping_all = []
for row in shopping_words:
    for r in row:
        if r not in shopping_all:
            shopping_all.append(r)
        else:
            pass

### 로지스틱 회귀

In [125]:
X = shopping_words
y = shopping['평점']

# 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [126]:
# 데이터 백터화

def identity_tokenizer(text):
    return text

vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer,lowercase=False)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

C:\Users\jhkim\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [127]:
# 모델(로지스틱 회귀)

model = LogisticRegression()
model.fit(X_train_vec, y_train)

C:\Users\jhkim\miniconda3\envs\ds_study\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [128]:
y_pred = model.predict(X_test_vec)
accuracy_score(y_test,y_pred)

0.52525

## 나이브 베이즈

In [140]:
# 나이브 베이즈 사용
from sklearn.naive_bayes import MultinomialNB

# alpha : 스무딩
# 훈련에 주로 사용된 데이터가 들어왔을 땐 분류기가 잘 작동하지만, 
# 훈련 데이터에 없던 값이 들어오거나 이상값이 들어올 경우 그에 대한 우도가 0이 되어, 
# 정상적인 분류가 되지 않을 때

naive = MultinomialNB(alpha=0.01)
naive.fit(X_train_vec, y_train)

MultinomialNB(alpha=0.01)

In [141]:
naive_pred = naive.predict(X_test_vec)
accuracy_score(y_test, naive_pred)

0.484

## 샴푸 데이터 예측

In [131]:
shampoo_data[shampoo_data['평점'] == 3]

,리뷰,평점,긍정부정,리뷰길이
2351,항상 쓰던 거샀어요,3,y,10
2352,미끌거림 없고 요순하고 좋아요,3,y,15
2353,조금 먀 써도 거품은 많이 나네요 머리카락 덜 빠지면 좋겠네요,3,y,30
2354,사용 전이요 감사합니다,3,n,13
2355,항상 사용하던 제품이에요,3,y,13
...,...,...,...,...
4846,몇 번 써봐야 알겠지만 거품도 잘 나는 편이고 감고 나면 시원해요 샘플도 사은품도 ...,3,y,69
4847,향는 괜찮아요 다른 건 잘 모르겠어요,3,n,19
4848,가격도 저렴하고 품질도 좋아요,3,y,16
4849,탈모엔 크게 도움이 되지 않았어요 좀 가렵기도 하구요 두 번 이상 세정을 해야 각질...,3,n,55


In [132]:
without_3 = all_words[:2351] + all_words[4851:]

In [133]:
len(without_3)

7779

### 말뭉치에 없는 단어 제거

In [135]:
for i, words in enumerate(without_3):
    for word in words:
        if word not in shopping_all:
            without_3[i].remove(word)

In [136]:
shampoo_X = without_3
shampoo_y = list(shampoo_data[shampoo_data['평점'] != 3]['평점'])

# 데이터 나누기
X_train_sham, X_test_sham, y_train_sham, y_test_sham = train_test_split(shampoo_X, shampoo_y, test_size=0.2, random_state=13)

In [137]:
vectorizer_sham = TfidfVectorizer(tokenizer=identity_tokenizer,lowercase=False)
X_train_vec_sham = vectorizer.fit_transform(X_train_sham)
X_test_vec_sham = vectorizer.transform(X_test_sham)

C:\Users\jhkim\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## 0921

In [22]:
shopping_data = pd.read_csv('./naver_shopping.csv')
shopping_data.rename(columns={'5':'평점','배공빠르고 굿':'리뷰'},inplace=True)
shopping_data

,평점,리뷰
0,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
1,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
2,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
3,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
4,2,비추합니다 계란 뒤집을 때 완전 불편해요 ㅠㅠ 코팅도 묻어나고 보기엔 예쁘고 실용적...
...,...,...
199994,2,장마라그런가!!! 달지않아요
199995,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199996,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199997,5,넘이쁘고 쎄련되보이네요~


In [23]:
shopping_data['평점'].value_counts()

5    81176
2    63989
1    36048
4    18786
Name: 평점, dtype: int64

In [24]:
s_5 = shopping_data[shopping_data['평점'] == 5]
score_5 = list(np.random.choice(s_5['리뷰'], 10000, replace=False))
s_4 = shopping_data[shopping_data['평점'] == 4]
score_4 = list(np.random.choice(s_4['리뷰'], 10000, replace=False))
s_2 = shopping_data[shopping_data['평점'] == 2]
score_2 = list(np.random.choice(s_2['리뷰'], 10000, replace=False))
s_1 = shopping_data[shopping_data['평점'] == 1]
score_1 = list(np.random.choice(s_1['리뷰'], 10000, replace=False))
review = score_5 + score_4 + score_2 + score_1

In [25]:
scores = []
for i in range(40000):
    if i < 10000:
        scores.append(5)
    elif i < 20000:
        scores.append(4)
    elif i < 30000:
        scores.append(2)
    else:
        scores.append(1)

In [26]:
shopping = pd.DataFrame({'리뷰':review,'평점':scores})
shopping.head()

,리뷰,평점
0,약해보이지만 나쁘지 않아요. 마감이 거친 부분이 있으나 살짝 갈아서 쓸려고요.ㅎㅎ,5
1,차로 끓여먹으니 구수하니 좋네요~,5
2,잘쓸게요 . 즐캠요,5
3,잘 받았습니다. 효과 있었으면 좋겠습ㄴ다,5
4,저렴하게 사서 좋아요~ 엄마께 선물로 드렸는데 좋아하세여^^,5


In [27]:
okt = Okt()

In [29]:
import re
def test(s):
    hangul = re.compile('[^ 가-힣0-9]+') # 한글과 띄어쓰기를 제외한 모든 글자
  # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
    result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    result_2 = hangul.findall(s) # 정규식에 일치되는 부분을 리스트 형태로 저장
    return result

In [30]:
for idx, row in shopping.iterrows():
    shopping.loc[idx, '리뷰'] = test(row['리뷰'])


In [31]:
shopping.head(5)

,리뷰,평점
0,약해보이지만 나쁘지 않아요 마감이 거친 부분이 있으나 살짝 갈아서 쓸려고요,5
1,차로 끓여먹으니 구수하니 좋네요,5
2,잘쓸게요 즐캠요,5
3,잘 받았습니다 효과 있었으면 좋겠습다,5
4,저렴하게 사서 좋아요 엄마께 선물로 드렸는데 좋아하세여,5


In [33]:
for idx, row in shopping.iterrows():
    if row['평점'] == 5 or row['평점'] == 4:
        shopping.loc[idx,'긍정부정'] = 1
    else:
        shopping.loc[idx, '긍정부정'] = 0

In [34]:
words_cnt = {}

for idx, row in shopping.iterrows():
    
    sentence = okt.pos(row['리뷰'],stem=True)
    for each in sentence:
        if each[1] not in ['Josa','Conjunction','Suffix']:
            if each[0] not in words_cnt:
                words_cnt[each[0]] = 1
            else:
                words_cnt[each[0]] += 1

In [35]:
# 긍정(5,4) 단어 빈도수
words_cnt_pos = {}

for idx, row in shopping[:20000].iterrows():
    
    sentence = okt.pos(row['리뷰'],stem=True)
    for each in sentence:
        if each[1] not in ['Josa','Conjunction','Suffix']:
            if each[0] not in words_cnt_pos:
                words_cnt_pos[each[0]] = 1
            else:
                words_cnt_pos[each[0]] += 1

In [36]:
# 부정(1,2) 단어 빈도수
words_cnt_neg = {}

for idx, row in shopping[20000:].iterrows():
    
    sentence = okt.pos(row['리뷰'],stem=True)
    for each in sentence:
        if each[1] not in ['Josa','Conjunction','Suffix']:
            if each[0] not in words_cnt_neg:
                words_cnt_neg[each[0]] = 1
            else:
                words_cnt_neg[each[0]] += 1

In [37]:
words_cnt = sorted(words_cnt.items(), key=lambda x: x[1])[::-1]
words_cnt_pos = sorted(words_cnt_pos.items(), key=lambda x:x[1])[::-1]
words_cnt_neg = sorted(words_cnt_neg.items(), key=lambda x:x[1])[::-1]

In [38]:
words_cnt

[('하다', 19797),
 ('좋다', 11997),
 ('배송', 5705),
 ('너무', 5668),
 ('같다', 4676),
 ('있다', 4589),
 ('안', 4097),
 ('자다', 4094),
 ('먹다', 3420),
 ('않다', 3353),
 ('없다', 3286),
 ('재다', 3084),
 ('구매', 3027),
 ('이다', 2921),
 ('쓰다', 2901),
 ('사용', 2848),
 ('되다', 2800),
 ('오다', 2763),
 ('사다', 2704),
 ('보다', 2635),
 ('그냥', 2566),
 ('제품', 2406),
 ('가격', 2356),
 ('생각', 2330),
 ('잘', 2313),
 ('빠르다', 2242),
 ('좀', 2193),
 ('다', 2187),
 ('이', 2105),
 ('받다', 2079),
 ('주문', 2012),
 ('맛', 1901),
 ('아니다', 1895),
 ('못', 1789),
 ('많이', 1735),
 ('더', 1705),
 ('한', 1645),
 ('것', 1617),
 ('별로', 1604),
 ('때', 1596),
 ('사이즈', 1561),
 ('들다', 1486),
 ('써다', 1460),
 ('만족하다', 1448),
 ('괜찮다', 1426),
 ('개', 1416),
 ('포장', 1369),
 ('맛있다', 1360),
 ('편하다', 1337),
 ('작다', 1332),
 ('크다', 1326),
 ('안되다', 1304),
 ('맞다', 1292),
 ('그렇다', 1288),
 ('상품', 1276),
 ('거', 1251),
 ('요', 1211),
 ('저렴하다', 1180),
 ('부분', 1173),
 ('냄새', 1142),
 ('싸다', 1142),
 ('다른', 1100),
 ('이쁘다', 1100),
 ('버리다', 1083),
 ('나다', 1074),
 ('보내다', 1058),
 ('모르다'

In [39]:
words_cnt_pos

[('좋다', 9878),
 ('하다', 8955),
 ('배송', 3261),
 ('자다', 2777),
 ('있다', 2473),
 ('같다', 2406),
 ('먹다', 2180),
 ('재다', 2095),
 ('너무', 2013),
 ('빠르다', 1954),
 ('구매', 1772),
 ('잘', 1510),
 ('사용', 1436),
 ('가격', 1412),
 ('만족하다', 1292),
 ('않다', 1282),
 ('보다', 1281),
 ('쓰다', 1274),
 ('이다', 1259),
 ('사다', 1214),
 ('되다', 1192),
 ('맛있다', 1185),
 ('편하다', 1116),
 ('생각', 1097),
 ('좀', 1031),
 ('제품', 1009),
 ('괜찮다', 1002),
 ('주문', 991),
 ('들다', 940),
 ('더', 934),
 ('없다', 899),
 ('이쁘다', 893),
 ('안', 870),
 ('저렴하다', 866),
 ('오다', 859),
 ('받다', 859),
 ('것', 855),
 ('맛', 853),
 ('때', 826),
 ('많이', 793),
 ('이', 769),
 ('예쁘다', 760),
 ('사이즈', 752),
 ('좋아하다', 703),
 ('써다', 702),
 ('감사하다', 689),
 ('아직', 675),
 ('포장', 667),
 ('아주', 665),
 ('다', 665),
 ('깔끔하다', 658),
 ('크다', 633),
 ('아이', 611),
 ('조금', 605),
 ('또', 592),
 ('거', 574),
 ('맘', 549),
 ('한', 526),
 ('구입', 522),
 ('넘다', 509),
 ('잘쓰다', 505),
 ('튼튼하다', 489),
 ('정말', 489),
 ('딱', 471),
 ('가성', 462),
 ('용', 458),
 ('작다', 444),
 ('선물', 444),
 ('디자인', 440),
 

In [40]:
words_cnt_neg

[('하다', 10842),
 ('너무', 3655),
 ('안', 3227),
 ('배송', 2444),
 ('없다', 2387),
 ('그냥', 2290),
 ('같다', 2270),
 ('좋다', 2119),
 ('있다', 2116),
 ('않다', 2071),
 ('오다', 1904),
 ('이다', 1662),
 ('쓰다', 1627),
 ('되다', 1608),
 ('아니다', 1543),
 ('다', 1522),
 ('사다', 1490),
 ('못', 1464),
 ('별로', 1453),
 ('사용', 1412),
 ('제품', 1397),
 ('보다', 1354),
 ('이', 1336),
 ('자다', 1317),
 ('구매', 1255),
 ('먹다', 1240),
 ('생각', 1233),
 ('받다', 1220),
 ('안되다', 1163),
 ('좀', 1162),
 ('한', 1119),
 ('개', 1063),
 ('맛', 1048),
 ('주문', 1021),
 ('버리다', 1007),
 ('그렇다', 997),
 ('재다', 989),
 ('가격', 944),
 ('많이', 942),
 ('부분', 893),
 ('작다', 888),
 ('맞다', 875),
 ('반품', 875),
 ('상품', 848),
 ('냄새', 827),
 ('보내다', 824),
 ('사이즈', 809),
 ('잘', 803),
 ('요', 791),
 ('더', 771),
 ('때', 770),
 ('것', 762),
 ('나다', 760),
 ('써다', 758),
 ('싸다', 722),
 ('다른', 720),
 ('포장', 702),
 ('모르다', 701),
 ('크다', 693),
 ('하나', 679),
 ('거', 677),
 ('제', 648),
 ('시키다', 620),
 ('진짜', 614),
 ('두', 610),
 ('불편하다', 607),
 ('저', 591),
 ('떨어지다', 584),
 ('아쉽다', 581),
 (

In [41]:
remove_words = ['하다','배송','같다','있다','이다','사용','사다','되다','좀']

In [42]:
# 일단 제거하지 말고 로지스틱 회귀만 해보기
# 해보고 성능이 어제보다 좋으면 나이브 베이즈도 하기

In [43]:
shopping_words = []

for idx, row in shopping.iterrows():
    
    sentence = okt.pos(row['리뷰'],stem=True)
    words = []
    for each in sentence:
        if each[1] not in ['Josa','Conjunction','Suffix']:
            words.append(each[0])
    shopping_words.append(words)

In [44]:
X = shopping_words
y = shopping['평점']

# 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

# 데이터 백터화

def identity_tokenizer(text):
    return text

vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer,lowercase=False)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 모델(로지스틱 회귀)

model = LogisticRegression()
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
accuracy_score(y_test,y_pred)

C:\Users\jhkim\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\jhkim\miniconda3\envs\ds_study\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.53375

In [45]:
# 별차이 없음

In [46]:
# 빈도수로 단어 제거 후 로지스틱 회귀

In [47]:
for i, words in enumerate(shopping_words):
    for word in words:
        if word in remove_words:
            shopping_words[i].remove(word)

In [48]:
shopping_words

[['약', '해보다', '나쁘다', '않다', '마감', '거치다', '부분', '살짝', '갈다', '쓸리다'],
 ['차로', '끓이다', '먹다', '구수하다', '좋다'],
 ['잘쓰다', '즐캠'],
 ['자다', '받다', '효과', '좋다', '습', '다'],
 ['저렴하다', '사서', '좋다', '엄마', '선물', '드리다', '좋아하다'],
 ['재다', '쿠폰', '저렴하다', '구입', '감사하다'],
 ['또', '주문', '아기', '목욕', '때', '자다', '쓰다'],
 ['좋다', '국민', '치', '발기', '무게', '감', '있다'],
 ['원', '단', '이브', '드럽다', '신축', '좋다'],
 ['두껍다', '저렴하다', '먼지', '적다'],
 ['메이커', '이름', '붙이다', '29000원', '짜다', '리', '똑같다'],
 ['저렴하다', '국산', '귀리', '자다', '구입', '한', '것'],
 ['흰색', '구입', '이쁘다'],
 ['피부', '땡기다', '없이', '괜찮다'],
 ['이전',
  '구매',
  '것',
  '정말',
  '튼튼하다',
  '좋다',
  '이번',
  '선물',
  '용',
  '많이',
  '빠르다',
  '안전하다',
  '보내다',
  '감사하다'],
 ['재다', '존맛', '탱탱', '구리', '존맛', '탱탱', '탱'],
 ['재다',
  '저',
  '범',
  '먹다',
  '때',
  '괜찮다',
  '번',
  '네모',
  '햄',
  '살짝',
  '비리다',
  '그래도',
  '맛있다',
  '만족하다'],
 ['저렴하다', '가격', '잘쓰다', '욤'],
 ['두께', '두껍다', '방', '하다', '너무', '좋다', '안막', '잘', '따다', '해'],
 ['빠르다',
  '포장',
  '자다',
  '되어다',
  '오다',
  '아이',
  '엄청',
  '좋아하다',
  '듯',
  '18',
  '개',


In [49]:
X = shopping_words
y = shopping['평점']

# 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

# 데이터 백터화

def identity_tokenizer(text):
    return text

vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer,lowercase=False)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 모델(로지스틱 회귀)

model = LogisticRegression()
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
accuracy_score(y_test,y_pred)

C:\Users\jhkim\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\jhkim\miniconda3\envs\ds_study\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.52975

In [50]:
# 나이브 베이즈 사용
from sklearn.naive_bayes import MultinomialNB

# alpha : 스무딩
# 훈련에 주로 사용된 데이터가 들어왔을 땐 분류기가 잘 작동하지만, 
# 훈련 데이터에 없던 값이 들어오거나 이상값이 들어올 경우 그에 대한 우도가 0이 되어, 
# 정상적인 분류가 되지 않을 때

naive = MultinomialNB(alpha=0.01)
naive.fit(X_train_vec, y_train)

naive_pred = naive.predict(X_test_vec)
accuracy_score(y_test, naive_pred)

0.484625

In [57]:
# 네이버 쇼핑 리뷰 데이터를 긍정(5,4)과 부정(1,2)에 비슷한 비율에 등장 빈도수가 높은 단어 8개 정도 빼고 
# 로지스틱 회귀한 결과 51%나와서 이 방법도 잘되지 않는 것 같습니다...

In [58]:
# 빈도 수 뺴고 긍정부정 추가해서 점수 예측

In [52]:
shopping.loc[shopping['평점'] == 5, '긍정부정'] = 1
shopping.loc[shopping['평점'] == 4, '긍정부정'] = 1
shopping.loc[shopping['평점'] == 2, '긍정부정'] = 0
shopping.loc[shopping['평점'] == 1, '긍정부정'] = 0

In [53]:
shopping['긍정부정'] = shopping['긍정부정'].astype('int')
shopping.head()

,리뷰,평점,긍정부정
0,약해보이지만 나쁘지 않아요 마감이 거친 부분이 있으나 살짝 갈아서 쓸려고요,5,1
1,차로 끓여먹으니 구수하니 좋네요,5,1
2,잘쓸게요 즐캠요,5,1
3,잘 받았습니다 효과 있었으면 좋겠습다,5,1
4,저렴하게 사서 좋아요 엄마께 선물로 드렸는데 좋아하세여,5,1


In [101]:
shopping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   리뷰      40000 non-null  object 
 1   평점      40000 non-null  int64  
 2   긍정부정    40000 non-null  int32  
 3   형태소     40000 non-null  object 
 4   리뷰길이    40000 non-null  float64
dtypes: float64(1), int32(1), int64(1), object(2)
memory usage: 1.4+ MB


In [54]:
shopping['형태소'] = shopping_words

In [69]:
for idx, row in shopping.iterrows():
    shopping.loc[idx,'리뷰길이'] = len(row['리뷰'])

In [70]:
X = shopping[['형태소','긍정부정','리뷰길이']]
y = shopping['평점']

# 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [76]:
# 데이터 백터화
from scipy.sparse import hstack

def identity_tokenizer(text):
    return text

vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer,lowercase=False,max_features=2000)
X_train_vec = vectorizer.fit_transform(X_train['형태소'])
X_test_vec = vectorizer.transform(X_test['형태소'])

# ['긍정부정'] 특성을 희소 행렬 형태로 변환
X_train_sentiment = X_train[['긍정부정']].values
X_test_sentiment = X_test[['긍정부정']].values

# ['리뷰길이'] 특성을 희소 행렬 형태로 변환
X_train_sentiment_2 = X_train[['리뷰길이']].values
X_test_sentiment_2 = X_test[['리뷰길이']].values


# 희소 행렬 형태로 변환된 ['긍정부정'] 특성을 TF-IDF 행렬과 결합
X_train_combined = hstack([X_train_vec, X_train_sentiment,X_train_sentiment_2])
X_test_combined = hstack([X_test_vec, X_test_sentiment,X_test_sentiment_2])

C:\Users\jhkim\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [57]:
# 모델(로지스틱 회귀)

model = LogisticRegression()
model.fit(X_train_combined, y_train)

y_pred = model.predict(X_test_combined)
accuracy_score(y_test,y_pred)

C:\Users\jhkim\miniconda3\envs\ds_study\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.607375

In [77]:
X_train_combined.shape, X_test_combined.shape

((32000, 2002), (8000, 2002))

In [78]:
# 모델(로지스틱 회귀) + 리뷰길이

model_2 = LogisticRegression()
model_2.fit(X_train_combined, y_train)

y_pred_2 = model_2.predict(X_test_combined)
accuracy_score(y_test,y_pred_2)

C:\Users\jhkim\miniconda3\envs\ds_study\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.610875

In [79]:
# 나이브 베이즈 사용
from sklearn.naive_bayes import MultinomialNB

# alpha : 스무딩
# 훈련에 주로 사용된 데이터가 들어왔을 땐 분류기가 잘 작동하지만, 
# 훈련 데이터에 없던 값이 들어오거나 이상값이 들어올 경우 그에 대한 우도가 0이 되어, 
# 정상적인 분류가 되지 않을 때

naive = MultinomialNB(alpha=0.01)
naive.fit(X_train_combined, y_train)

naive_pred = naive.predict(X_test_combined)
accuracy_score(y_test, naive_pred)

0.5915

In [60]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

rf.fit(X_train_combined, y_train)
y_pred = rf.predict(X_test_combined)
accuracy_score(y_test,y_pred)

0.601125

In [61]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)

gb.fit(X_train_combined, y_train)
y_pred = gb.predict(X_test_combined)
accuracy_score(y_test,y_pred)

0.600125

In [62]:
# scores = cross_val_score(gb, X_train_combined, y_train, cv=5, scoring='accuracy')
# scores

In [63]:
X_train_combined.shape

(32000, 17230)

In [64]:
X_train_vec.shape

(32000, 17229)

In [65]:
# 샴푸 데이터로 테스트

In [122]:
shampoo = pd.read_csv('./shampoo_final.csv')
shampoo

,리뷰,평점,긍정부정,리뷰길이
0,진짜 머리 감고 나면 너무 너무 간지러워 요상쾌함이 오래가지도 못하고 냄새도 빨리 나요,1,n,52
1,비듬 생겼어요 어쩐지간지럽더라 개나 샀는데 누구 주지도 못하고 아오,1,n,50
2,뚜껑 펌프 고장 났네요 아무리 눌러도 안 나와서 뚜껑 열고 사용하다 방치했어요,1,n,40
3,머리카락에 부담이 안 되고 머리 피부에 부담이 안 되니 좋죠,1,n,27
4,샴푸는 사용해보고 살수가 없으니 오랜 기간 동안 고민했고 리뷰들도 꼼꼼히 보았는데 ...,1,n,510
...,...,...,...,...
10274,단백질 샴푸 첫 구매입니다 대 넘어가니 탈모에만 신경이 쓰이고 그래서 탈모샴푸만 사...,5,y,222
10275,개인적으로 단백질 본드 샴푸세트를 몇 년 동안 사용 중인데 머릿결이 윤기가 돌아 요...,5,y,223
10276,두피가 예민하고 각질이 많아 요티트리 샴푸가 좋다고 해서 구매해봤어요 성분이 좋아서...,5,y,126
10277,두피에 좋은 제품들이 거품이 많이 안 난다는데 이 제품은 거품 진짜 풍성하고 세정력...,5,y,176


In [123]:
shampoo_words = []

for idx, row in shampoo.iterrows():
    sentence = okt.pos(row['리뷰'],stem=True)
    words = []
    for each in sentence:
        if each[1] not in ['Josa','Conjunction','Suffix']:
            words.append(each[0])
    shampoo_words.append(words)

In [124]:
shampoo['형태소'] = shampoo_words

In [125]:
shampoo

,리뷰,평점,긍정부정,리뷰길이,형태소
0,진짜 머리 감고 나면 너무 너무 간지러워 요상쾌함이 오래가지도 못하고 냄새도 빨리 나요,1,n,52,"[진짜, 머리, 감고, 나다, 너무, 너무, 간지럽다, 요, 상쾌하다, 오래, 가지..."
1,비듬 생겼어요 어쩐지간지럽더라 개나 샀는데 누구 주지도 못하고 아오,1,n,50,"[비듬, 생기다, 어쩐지, 간지럽다, 개, 사다, 누구, 주지, 못, 아오]"
2,뚜껑 펌프 고장 났네요 아무리 눌러도 안 나와서 뚜껑 열고 사용하다 방치했어요,1,n,40,"[뚜껑, 펌프, 고장, 나다, 아무리, 누르다, 안, 나오다, 뚜껑, 열다, 사용,..."
3,머리카락에 부담이 안 되고 머리 피부에 부담이 안 되니 좋죠,1,n,27,"[머리카락, 부담, 안, 되다, 머리, 피부, 부담, 안, 되다, 좋다]"
4,샴푸는 사용해보고 살수가 없으니 오랜 기간 동안 고민했고 리뷰들도 꼼꼼히 보았는데 ...,1,n,510,"[샴푸, 사용, 해보다, 살수, 없다, 오랜, 기간, 동안, 고민, 하다, 리뷰, ..."
...,...,...,...,...,...
10274,단백질 샴푸 첫 구매입니다 대 넘어가니 탈모에만 신경이 쓰이고 그래서 탈모샴푸만 사...,5,y,222,"[단백질, 샴푸, 첫, 구매, 이다, 대다, 넘어가다, 탈모, 신경, 쓰이다, 그래..."
10275,개인적으로 단백질 본드 샴푸세트를 몇 년 동안 사용 중인데 머릿결이 윤기가 돌아 요...,5,y,223,"[개인, 단백질, 본드, 샴푸, 세트, 몇, 년, 동안, 사용, 중, 머릿결, 윤기..."
10276,두피가 예민하고 각질이 많아 요티트리 샴푸가 좋다고 해서 구매해봤어요 성분이 좋아서...,5,y,126,"[두피, 예민하다, 각질, 많다, 요, 티, 트리, 샴푸, 좋다, 하다, 구매, 해..."
10277,두피에 좋은 제품들이 거품이 많이 안 난다는데 이 제품은 거품 진짜 풍성하고 세정력...,5,y,176,"[두피, 좋다, 제품, 거품, 많이, 안, 나다, 이, 제품, 거품, 진짜, 풍, ..."


In [126]:
shampoo.loc[shampoo['긍정부정'] == 'n','긍정부정'] = 0 
shampoo.loc[shampoo['긍정부정'] == 'y','긍정부정'] = 1 

In [127]:
shampoo

,리뷰,평점,긍정부정,리뷰길이,형태소
0,진짜 머리 감고 나면 너무 너무 간지러워 요상쾌함이 오래가지도 못하고 냄새도 빨리 나요,1,0,52,"[진짜, 머리, 감고, 나다, 너무, 너무, 간지럽다, 요, 상쾌하다, 오래, 가지..."
1,비듬 생겼어요 어쩐지간지럽더라 개나 샀는데 누구 주지도 못하고 아오,1,0,50,"[비듬, 생기다, 어쩐지, 간지럽다, 개, 사다, 누구, 주지, 못, 아오]"
2,뚜껑 펌프 고장 났네요 아무리 눌러도 안 나와서 뚜껑 열고 사용하다 방치했어요,1,0,40,"[뚜껑, 펌프, 고장, 나다, 아무리, 누르다, 안, 나오다, 뚜껑, 열다, 사용,..."
3,머리카락에 부담이 안 되고 머리 피부에 부담이 안 되니 좋죠,1,0,27,"[머리카락, 부담, 안, 되다, 머리, 피부, 부담, 안, 되다, 좋다]"
4,샴푸는 사용해보고 살수가 없으니 오랜 기간 동안 고민했고 리뷰들도 꼼꼼히 보았는데 ...,1,0,510,"[샴푸, 사용, 해보다, 살수, 없다, 오랜, 기간, 동안, 고민, 하다, 리뷰, ..."
...,...,...,...,...,...
10274,단백질 샴푸 첫 구매입니다 대 넘어가니 탈모에만 신경이 쓰이고 그래서 탈모샴푸만 사...,5,1,222,"[단백질, 샴푸, 첫, 구매, 이다, 대다, 넘어가다, 탈모, 신경, 쓰이다, 그래..."
10275,개인적으로 단백질 본드 샴푸세트를 몇 년 동안 사용 중인데 머릿결이 윤기가 돌아 요...,5,1,223,"[개인, 단백질, 본드, 샴푸, 세트, 몇, 년, 동안, 사용, 중, 머릿결, 윤기..."
10276,두피가 예민하고 각질이 많아 요티트리 샴푸가 좋다고 해서 구매해봤어요 성분이 좋아서...,5,1,126,"[두피, 예민하다, 각질, 많다, 요, 티, 트리, 샴푸, 좋다, 하다, 구매, 해..."
10277,두피에 좋은 제품들이 거품이 많이 안 난다는데 이 제품은 거품 진짜 풍성하고 세정력...,5,1,176,"[두피, 좋다, 제품, 거품, 많이, 안, 나다, 이, 제품, 거품, 진짜, 풍, ..."


In [128]:
without_3 = shampoo[shampoo['평점'] != 3]

In [129]:
shopping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   리뷰      40000 non-null  object 
 1   평점      40000 non-null  int64  
 2   긍정부정    40000 non-null  int32  
 3   형태소     40000 non-null  object 
 4   리뷰길이    40000 non-null  float64
dtypes: float64(1), int32(1), int64(1), object(2)
memory usage: 1.4+ MB


In [130]:
without_3['긍정부정'] = without_3['긍정부정'].astype('int')
without_3['리뷰길이'] = without_3['리뷰길이'].astype('float64')

C:\Users\jhkim\AppData\Local\Temp\ipykernel_19408\3389143273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  without_3['긍정부정'] = without_3['긍정부정'].astype('int')
C:\Users\jhkim\AppData\Local\Temp\ipykernel_19408\3389143273.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  without_3['리뷰길이'] = without_3['리뷰길이'].astype('float64')


In [131]:
without_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7779 entries, 0 to 10278
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   리뷰      7779 non-null   object 
 1   평점      7779 non-null   int64  
 2   긍정부정    7779 non-null   int32  
 3   리뷰길이    7779 non-null   float64
 4   형태소     7779 non-null   object 
dtypes: float64(1), int32(1), int64(1), object(2)
memory usage: 334.3+ KB


In [132]:
X = without_3[['형태소','긍정부정','리뷰길이']]
y = without_3['평점']

# 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [134]:
# 데이터 백터화

vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer,lowercase=False,max_features=2000)
X_train_vec = vectorizer.fit_transform(X_train['형태소'])
X_test_vec = vectorizer.transform(X_test['형태소'])

# ['긍정부정'] 특성을 희소 행렬 형태로 변환
X_train_sentiment = X_train[['긍정부정']].values
X_test_sentiment = X_test[['긍정부정']].values

# ['리뷰길이'] 특성을 희소 행렬 형태로 변환
X_train_sentiment_2 = X_train[['리뷰길이']].values
X_test_sentiment_2 = X_test[['리뷰길이']].values


# 희소 행렬 형태로 변환된 ['긍정부정'] 특성을 TF-IDF 행렬과 결합
X_train_combined = hstack([X_train_vec, X_train_sentiment,X_train_sentiment_2])
X_test_combined = hstack([X_test_vec, X_test_sentiment,X_test_sentiment_2])

C:\Users\jhkim\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [135]:
X_train_combined.shape, X_test_combined.shape

((6223, 2002), (1556, 2002))

In [136]:
# 모델(로지스틱 회귀)

y_pred_3 = model_2.predict(X_test_combined)
accuracy_score(y_test,y_pred_3)

0.36568123393316193

In [137]:
# 데이터 백터화

vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer,lowercase=False,max_features=2000)
X_vec = vectorizer.fit_transform(X['형태소'])


# ['긍정부정'] 특성을 희소 행렬 형태로 변환
X_sentiment = X[['긍정부정']].values


# ['리뷰길이'] 특성을 희소 행렬 형태로 변환
X_sentiment_2 = X[['리뷰길이']].values



# 희소 행렬 형태로 변환된 ['긍정부정'] 특성을 TF-IDF 행렬과 결합
X_combined = hstack([X_vec, X_sentiment,X_sentiment_2])


C:\Users\jhkim\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [138]:
y_pred_3 = model_2.predict(X_combined)
accuracy_score(y,y_pred_3)

0.3608432960534773

In [140]:
from sklearn.metrics import classification_report

print(classification_report(y, y_pred_3))

              precision    recall  f1-score   support

           1       0.43      0.29      0.35      1183
           2       0.43      0.64      0.51      1168
           4       0.31      0.37      0.34      2705
           5       0.35      0.26      0.30      2723

    accuracy                           0.36      7779
   macro avg       0.38      0.39      0.37      7779
weighted avg       0.36      0.36      0.35      7779

